Function to preprocess data

In [3]:
SEED = 2022
#!/usr/bin/env python
import sys
import os
sys.path.append('src/')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import argparse
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer, QuantileTransformer
from sklearn.decomposition import PCA
import logging
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import (
    TimeSeriesSplit,
    cross_val_score,
    cross_validate,
    train_test_split,
)
import time
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
import lightgbm as lgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

In [13]:
#Function to prepare X_train, y_train, X_test, y_test
def data_prep(df,target_variable):
    features = df.columns.to_list()
    features.remove(target_variable)
    X = df[features]
    y = df[[target_variable]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)
    return X_train, X_test, y_train, y_test
    

In [4]:
def fit_processor(X_train, target_variable):
    """
    Applies Simple Imputer to Categorical Features
    Applies One Hot Encoding to Categorical Features
    Applies Quantile Scaling to Numeric Features
    Returns and writes pickle file of the complete preprocessor

    Parameters
    ----------
    X_train : numpy
        dataset in NumPy format
    Will create lists of numeric and categorical 
        variables using X_train df

    Returns
    -------
    preprocessor : sklearn.Preprocessor
        sklearn preprocessor fit on the training set
    """
    
    features = X_train.columns.to_list()
    df_numeric_features = X_train.select_dtypes(include='number')
    df_categorical_features = X_train.select_dtypes(include='object')

    numeric_features = df_numeric_features.columns.to_list()
    categorical_features = df_categorical_features.columns.to_list()
    
    pipe_num = Pipeline([
        ('impute', SimpleImputer(strategy='mean')),
        ('scaler',  MinMaxScaler())
    ])
    pipe_cat = Pipeline([
        ('impute', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ]) 
    preprocessor = ColumnTransformer([
        ('num', pipe_num, numeric_features),
        ('cat', pipe_cat, categorical_features)
    ])

    norm_X_train = preprocessor.fit(X_train)

    return preprocessor

In [18]:
#import argparse
#parser = argparse.ArgumentParser()
#parser.add_argument('--file', type=str, required=True, help='Path to the CSV file')
#args = parser.parse_args()
#train_df = pd.read_csv(args.file)
#Run the following command on cmd using your file name
#python your_script.py --file result.csv 

train_df = pd.read_csv('result.csv') #read your dataset
target_variable='validation_loss' #define your target variable

usage: ipykernel_launcher.py [-h] --file FILE
ipykernel_launcher.py: error: the following arguments are required: --file


SystemExit: 2

In [8]:
X_train, X_test, y_train, y_test = data_prep(train_df,target_variable)
processor = fit_processor(train_df,target_variable)
norm_X_train = processor.transform(X_train)
norm_X_test = processor.transform(X_test)